# Wealthfront Take-Home

This notebook is a walk through of code solution for the Sr. Data Science interview homework assignment.

<b>Author:</b> Hannah Jones

<b>Date:</b> 2023-12-02

## Env Set Up

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
